In [1]:
import sympy as sp
from sympy import sin, cos, pi
from sympy.interactive import printing
import pickle
import numpy as np
import scipy as sc
import scipy.interpolate
from scipy.integrate import odeint
import matplotlib.pyplot as pl

import symbtools as st
import symbtools.modeltools as mt
import symbtools.noncommutativetools as nct

In [3]:
t = sp.Symbol('t')
Np = 1
Nq = 1
n = Np + Nq
pp = st.symb_vector("p1:{0}".format(Np+1))
qq = st.symb_vector("q1:{0}".format(Nq+1))
aa = st.symb_vector("a1:{0}".format(Nq+1))
ww = st.symb_vector("w1:{0}".format(Nq+1))

ttheta = st.row_stack(pp, qq) ##:T
tthetad = st.time_deriv(ttheta, ttheta) ##:T
tthetadd = st.time_deriv(ttheta, ttheta, order=2) ##:T
st.make_global(ttheta, tthetad)

In [4]:
params = sp.symbols('m1, m2, s2, g')
st.make_global(params)

tau1 = sp.Symbol("tau1")

In [6]:
#Einheitsvektoren
ex = sp.Matrix([1,0])
ey = sp.Matrix([0,1])


# Koordinaten der Schwerpunkte und Gelenke
S1 = ex*q1 # Schwerpunkt Wagen
G2 = S1 # Pendel-Gelenk

# Schwerpunkt des Pendels (Pendel zeigt für kleine Winkel nach #! OBEN)
S2 = G2 + mt.Rz(p1)*(ey)*s2

# Zeitableitungen der Schwerpunktskoordinaten
Sd1, Sd2  = st.col_split(st.time_deriv(st.col_stack(S1, S2), ttheta)) ##

In [7]:
# Energie
T_rot = 0 # (Punktmassenmodell)
T_trans = ( m1*Sd1.T*Sd1  +  m2*Sd2.T*Sd2 )/2

T = T_rot + T_trans[0]

V = m2*g*S2[1]

In [11]:
mod = mt.generate_symbolic_model(T, V, ttheta, [0, tau1])

In [12]:
mod.MM.simplify()
mod.MM

Matrix([
[      m2*s2**2, -m2*s2*cos(p1)],
[-m2*s2*cos(p1),        m1 + m2]])

In [16]:
mod.calc_coll_part_lin_state_eq(simplify=True)

In [35]:
mod.ff

Matrix([
[       pdot1],
[       qdot1],
[g*sin(p1)/s2],
[           0]])

In [19]:
xx = mod.x 
f = mod.ff 
G = mod.gg 
g1, = st.col_split(G)

In [26]:
eqp1 = sp.Matrix([0, 0, 0, 0 ]) ##:T
eqp2 = sp.Matrix([sp.pi, 0, 0, 0]) ##:T

# Probe:
assert f.subz(xx, eqp1) == sp.zeros(4,1)
assert f.subz(xx, eqp2) == sp.zeros(4,1)

In [29]:
# Parameterwerte
parameter_values = [(g, 9.81), (s2, .5)]
replm =  parameter_values + list(zip(xx, eqp1))

In [31]:
A1 = f.jacobian(xx).subs(replm) 
A1

Matrix([
[    0, 0, 1, 0],
[    0, 0, 0, 1],
[19.62, 0, 0, 0],
[    0, 0, 0, 0]])

In [33]:
b1 = G.subs(replm) 
b1

Matrix([
[  0],
[  0],
[2.0],
[  1]])

In [51]:
a = 1.2
b = 3.4

poles = np.r_[-1.5+a*1j, -1.5-a*1j, -1.3 + b*1j, -1.3 - b*1j] ##:
k1 = st.siso_place(A1, b1, poles)
print(k1)
# A1-b1@k1
import control as ctrl
F = ctrl.place(np.array(A1,dtype=float), np.array(b1,dtype=float), poles)
print("F", F)
np.linalg.eigvals(np.array(A1, dtype=float)- np.array(b1,dtype=float)@F)


Matrix([[-23.4259862385321], [2.49197247706422], [-4.05749235474006], [2.51498470948012]])
F [[23.42598624 -2.49197248  4.05749235 -2.51498471]]


array([-1.3+3.4j, -1.3-3.4j, -1.5+1.2j, -1.5-1.2j])

In [49]:
(A1-b1@k1.T).eigenvals()

{11.5442107450404 - 4.46351031349122e-63*I: 1,
 -5.20933544396007 + 4.94903402628429e-63*I: 1,
 -0.367437650540142 - 0.823407052335835*I: 1,
 -0.367437650540142 + 0.823407052335835*I: 1}